# IR spectroscopy

(sec:ir-tutorial)=

The infrared (IR) absorption process involves the absorption of light with low photon energy (from approx. 800 nm to approx. 1000 $\mu$m) which promotes the excitation of molecular vibrations. Of the entire IR photon energy range, the photons which excite fundamental vibrations of covalent bonds have energies in the range 2.5-25 $\mu$m {cite}`Norman2018, Centrone2015`. The absorbed photon energies (i.e. IR absorption peak positions) correspond to the molecular vibrational freqencies, while the IR intensities are related to the IR [linear absorption cross-section](sec:exact-response) {cite}`Norman2018`:
\begin{equation}
\sigma_k(\omega)= \frac{\pi}{3\epsilon_0c}\left(\frac{\partial \mu}{\partial Q_k}\right)^2\frac{\gamma_{k}}{(\omega_{k0}-\omega)^2+\gamma_{k}^2}\,.
\end{equation}
Here, $\mu$ is the molecular dipole moment of the electronic ground state, $Q_k$ is the normal coordinate corresponding to the vibrational mode $k$, $\omega$ is the angular frequency of the absorbed electromagnetic radiation, $\omega_{k0}$ is the anugular frequency difference between the excited vibrational state $|k\rangle$ and the ground state, and $\gamma_{k}$ is the half-width broadening associated with the inverse lifetime of $|k\rangle$. As can be seen from the equation, the IR absorption cross-section depends on the derivative of the dipole moment with respect to the normal coordinate $Q_k$. This means that the only modes which have non-zero IR absorption cross-sections are those associated with a change in the dipole moment {cite}`Siebert2007`.

These molecular vibrations constitute fingerprints of different functional groups present in the system and thus are important for chemical characterization and molecular identification.

The first step in an IR spectrum calculation is to determine the normal modes and frequencies. These are obtained from the eigenstates and eigenvalues of the Hessian matrix which collects all the second order derivatives of the energy ($E$) with respect to the nuclear coordinates. For the ground state, the Hessian can be determined [analytically](hessians:label), or numerically based on the analytical gradient. The [dipole moment gradient](dipole_mom_gradient:label), required to compute IR absorption cross-sections, can also be determined analytically. 


In [1]:
# Import section
import veloxchem as vlx
import numpy as np
import sys
import py3Dmol as p3d
from veloxchem.veloxchemlib import bohr_in_angstroms

In [2]:
# Define the molecule and basis set
molecule_string = """3

  O 0.000 0.000  0.000
  H 0.000 0.000  0.950
  H 0.896 0.000 -0.317"""
basis_set_label = 'sto-3g'

molecule = vlx.Molecule.from_xyz_string(molecule_string)
basis = vlx.MolecularBasis.read(molecule, basis_set_label)
ostream = vlx.OutputStream(sys.stdout)
ostream.print_block(molecule.get_string()) #TODO: show the molecular structure instead
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           O           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000000000                         
                           H           0.896000000000        0.000000000000       -0.317000000000                         
                                                                                                                          


```python
# Run the scf calculation
scf_settings = {}
method_settings = {} #{'xcfun':'b3lyp'}
scfdrv = vlx.ScfRestrictedDriver()
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
```


                                                                                                                              
                                                Self Consistent Field Driver Setup                                            
                                               ====================================                                           
                                                                                                                              
                       Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                       Initial Guess Model             : Superposition of Atomic Densities                                    
                       Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                       Max. Number of Iterations       : 50                                                                   
                       Max. Number of Error Vectors    : 10                                                                   
                       Convergence Threshold           : 1.0e-06                                                              
                       ERI Screening Scheme            : Cauchy Schwarz + Density                                             
                       ERI Screening Mode              : Dynamic                                                              
                       ERI Screening Threshold         : 1.0e-12                                                              
                       Linear Dependence Threshold     : 1.0e-06                                                              
                                                                                                                              
    * Info * Nuclear repulsion energy: 9.2514793147 a.u.                                                                      
                                                                                                                              
    * Info * Overlap matrix computed in 0.01 sec.                                                                             
                                                                                                                              
    * Info * Kinetic energy matrix computed in 0.00 sec.                                                                      
                                                                                                                              
    * Info * Nuclear potential matrix computed in 0.01 sec.                                                                   
                                                                                                                              
    * Info * Orthogonalization matrix computed in 0.06 sec.                                                                   
    ... 

Now we can calculate the Hessian 

```python
# Create the Hessian Driver and use it to compute an analytical SCF Hessian
hessian_settings = {}
hessian_drv = vlx.scfhessiandriver.ScfHessianDriver(scfdrv, scfdrv.comm, ostream)
hessian_drv.update_settings(method_settings, hessian_settings)
hessian_drv.compute(molecule, basis)
```

                                                                                                                              
                                                        SCF Hessian Driver                                                    
                                                       ====================                                                   
                                                                                                                              
                                       *** Time spent in Hessian calculation: 14.05 sec ***                                   
                                                                                                                              
                                                                                                                              



```python
hessian_drv.print_hessian(molecule)
```

                                               Numerical Hessian (Hartree/Bohr**2)                                            
                                              -------------------------------------                                           
                                                                                                                              
      Coord.       1 O(x)          1 O(y)          1 O(z)          2 H(x)          2 H(y)          2 H(z)          
                                                                                                                              
      1 O(x)       0.77341884      0.00000000     -0.22069579     -0.05793262     -0.00000000      0.05915298
      1 O(y)       0.00000000     -0.08462352     -0.00000000     -0.00000000      0.04250761      0.00000000
      1 O(z)      -0.22069502     -0.00000000      0.93182783     -0.07097734      0.00000000     -0.79549609
      2 H(x)      -0.05793235      0.00000000     -0.07097763      0.05352698     -0.00000000     -0.01075528
      2 H(y)      -0.00000000      0.04250764      0.00000000      0.00000000     -0.02940288     -0.00000000
      2 H(z)       0.05915305      0.00000000     -0.79549640     -0.01075545     -0.00000000      0.80288147
      3 H(x)      -0.71548655     -0.00000000      0.29167339      0.00440567      0.00000000     -0.04839771
      3 H(y)       0.00000000      0.04211595      0.00000000     -0.00000000     -0.01310475      0.00000000
      3 H(z)       0.16154197      0.00000000     -0.13633145      0.08173281      0.00000000     -0.00738538
                                                                                                                              
                                                                                                                              
      Coord.       3 H(x)          3 H(y)          3 H(z)          
                                                                                                                              
      1 O(x)      -0.71548621      0.00000000      0.16154281
      1 O(y)       0.00000000      0.04211592     -0.00000000
      1 O(z)       0.29167237      0.00000000     -0.13633174
      2 H(x)       0.00440537     -0.00000000      0.08173291
      2 H(y)      -0.00000000     -0.01310475      0.00000000
      2 H(z)      -0.04839760      0.00000000     -0.00738507
      3 H(x)       0.71108088      0.00000000     -0.24327568
      3 H(y)       0.00000000     -0.02901119     -0.00000000
      3 H(z)      -0.24327478     -0.00000000      0.14371683
                                                                                                                              
                                                                                                                              


The normal modes and frequencies are then determined by diagonalizing the Hessian. First, the transation and rotational modes mut be projected out, leaving 3N-6 (or 3N-5 for a linear molecule) vibrational modes. This is done under the hood with the help of geomeTRIC. 

The IR intensities are determined as discussed in more details [here](dipole_mom_gradient:label).


```python
hessian_drv.vibrational_analysis(molecule)
```

                                                       Vibrational Analysis                                                   
                                                      ======================                                                  
                                                                                                                              
                     Harmonic frequencies (in cm**-1), force constants (in mdyne/A), reduced masses (in amu),                 
                                                    IR intensities (in km/mol),                                               
                                             and Cartesian normal mode displacements.                                         
                                                                                                                              
                                                                                                                              
      Index:                        1                              2                              3               
      Frequency:                 1927.09                        4545.78                        4896.44            
      Force constant:            2.3750                         12.6970                        15.3533            
      Reduced mass:              1.0855                         1.0429                         1.0869             
      IR intensity:              15.0374                        30.3498                        12.8846            
      Normal mode:       X         Y         Z     |    X         Y         Z     |    X         Y         Z     |
      1 O              0.0587   -0.0000    0.0415  |  0.0397    0.0000    0.0274  | -0.0416   -0.0000    0.0595  |
      2 H             -0.7054    0.0000    0.0092  |  0.0299   -0.0000   -0.7000  |  0.0002   -0.0000   -0.7111  |
      3 H             -0.2266   -0.0000   -0.6677  | -0.6605    0.0000    0.2654  |  0.6594   -0.0000   -0.2331  |
                                                                                                                              
                                                                                                                              



```python
def add_broadening(list_ex_energy, list_osci_strength, line_profile='Lorentzian', line_param=10, step=10):
        x_min = np.amin(list_ex_energy) - 50
        x_max = np.amax(list_ex_energy) + 50
        x = np.arange(x_min, x_max, step)
        y = np.zeros((len(x)))
        #print(x)
        #print(y)

        # go through the frames and calculate the spectrum for each frame
        for xp in range(len(x)):
            for e, f in zip(list_ex_energy, list_osci_strength):
                if line_profile == 'Gaussian':
                    y[xp] += f * np.exp(-(
                        (e - x[xp]) / line_param)**2)
                elif line_profile == 'Lorentzian':
                    y[xp] += 0.5 * line_param * f / (np.pi * (
                        (x[xp] - e)**2 + 0.25 * line_param**2))
        return x, y
```


```python
# plot the IR spectrum
from matplotlib import pyplot as plt

plt.figure(figsize=(7,4))
# Spectrum from -CHF 
x1,y1 = hessian_drv.frequencies, hessian_drv.ir_intensities
x1i, y1i = add_broadening(x1,y1,line_profile='Gaussian',line_param=20,step=10)
plt.plot(x1i,y1i)
#plt.plot(x1,y1,'x')
plt.xlabel('wavenumber (cm**-1)')
plt.ylabel('IR intensity (km/mol)')
plt.title("Calculated IR sepctrum of the water molecule")
plt.tight_layout(); plt.show()
# TODO: add comparison to expriment
```


    
![png](../img/ir_raman/ir_output_9_0.png)
    


Let's now visualize the three vibrational normal modes. The two high frequency modes are OH stretching modes (one symmetryc and one asymmetric), while the lower energy mode is a bond angle bending.


```python
# To animate the normal mode we will need both the geometry and the displacements 
def get_normal_mode(molecule, normal_mode):
    elements = molecule.get_labels()
    coords = molecule.get_coordinates() * bohr_in_angstroms() # To transform from au to A
    natm = molecule.number_of_atoms()
    vib_xyz = "%d\n\n" % natm
    nm = normal_mode.reshape(natm, 3)
    for i in range(natm):
        # add coordinates:
        vib_xyz += elements[i] + " %15.7f %15.7f %15.7f " % (coords[i,0], coords[i,1], coords[i,2])
        # add displacements:
        vib_xyz += "%15.7f %15.7f %15.7f\n" % (nm[i,0], nm[i,1], nm[i,2])
    return vib_xyz
```


```python
vib_1 = get_normal_mode(molecule, hessian_drv.normal_modes[0])
vib_2 = get_normal_mode(molecule, hessian_drv.normal_modes[1])
vib_3 = get_normal_mode(molecule, hessian_drv.normal_modes[2])
```

In [3]:
vib_1 = """3

O       0.0000000       0.0000000       0.0000000       0.0587157      -0.0000000       0.0414896
H       0.0000000       0.0000000       0.9500000      -0.7053752       0.0000000       0.0091568
H       0.8960000       0.0000000      -0.3170000      -0.2266084      -0.0000000      -0.6677138"""

vib_2 = """3

O       0.0000000       0.0000000       0.0000000       0.0397286       0.0000000       0.0273807
H       0.0000000       0.0000000       0.9500000       0.0299225      -0.0000000      -0.7000252
H       0.8960000       0.0000000      -0.3170000      -0.6605266       0.0000000       0.2654166"""

vib_3 = """3

O       0.0000000       0.0000000       0.0000000      -0.0415549      -0.0000000       0.0594816
H       0.0000000       0.0000000       0.9500000       0.0001976      -0.0000000      -0.7110595
H       0.8960000       0.0000000      -0.3170000       0.6593957      -0.0000000      -0.2330811"""

In [4]:
print("This is the bending mode at 1927.09 cm-1.")
view = p3d.view(width=300, height=300)
view.addModel(vib_1, "xyz", {'vibrate': {'frames':10,'amplitude':0.75}})
view.setStyle({'stick':{}})
view.animate({'loop': 'backAndForth'})
view.zoomTo()
view.show()

This is the bending mode at 1927.09 cm-1.


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
print("This is the symmetric stretching mode at 4545.78 cm-1.")
view = p3d.view(width=300, height=300)
view.addModel(vib_2, "xyz", {'vibrate': {'frames':10,'amplitude':0.75}})
view.setStyle({'stick':{}})
view.animate({'loop': 'backAndForth'})
view.zoomTo()
view.show()

This is the symmetric stretching mode at 4545.78 cm-1.


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
print("This is the asymmetric stretching mode at 4896.44 cm-1.")
view = p3d.view(width=300, height=300)
view.addModel(vib_3, "xyz", {'vibrate': {'frames':10,'amplitude':0.75}})
view.setStyle({'stick':{}})
view.animate({'loop': 'backAndForth'})
view.zoomTo()
view.show()

This is the asymmetric stretching mode at 4896.44 cm-1.


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol